# Data Importing

In [1]:
# Importing modules
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression

# Read data into papers
data = pd.read_csv("/Users/atchoo/Google Drive/Research Assistant/Work With David Knight/Survey of the Incarcerated/TMPPoliticalSurveyFULL_ForDavid.csv", encoding = "ISO-8859-1")

# Print head
# print(data.columns)

In [2]:
# Insert an id index for future work
data["Survey_ID"] = data.index + 1

# Remove the columns
str_data = data[["Survey_ID", "explain_politics_changed_since_incarcerated",\
                 "explain_race_affects_politics", 'identifies_as_black', 'identifies_as_white',
                 "identifies_as_native", 'identifies_as_asian', 
                 'identifies_as_hawaiian_or_pac_islander', 'identifies_as_other_race',
                 'identifies_as_not_sure_of_race', 'identifies_as_hispanic_or_latinx',
                 "age", "gender"]]
# Print out the first rows of papers
str_data.columns = ["Survey_ID", "p_change", "r_effect", "black", "white", "native", "asian",
                   "hawaiian", "other_race", "unsure_race", "latinx", "age", "gender"]
str_data = str_data.dropna()

# Make Remove Stopword Functions

In [3]:
# Load the regular expression library and the nltk word library
import re
import nltk

# Create a function to remove nonsense words
words = set(nltk.corpus.words.words())
def clean_sent(sent):
    return " ".join(w for w in nltk.wordpunct_tokenize(sent) \
     if w.lower() in words or not w.isalpha())

strv_list = ["p_change", "r_effect"]
for strv in strv_list:
    # Remove punctuation & Convert the titles to lowercase
    str_data[strv] = str_data[strv].map(lambda x: re.sub(r'[^\w\s]', "", x)).map(lambda x: x.lower())
    # Remove nonsense words
    str_data[strv] = str_data[strv].apply(clean_sent)
    # Transform blank cells to NaN & Drop NaN
    str_data = str_data.replace(r'^\s*$', np.nan, regex=True).dropna()

In [4]:
str_data.head()

,Survey_ID,p_change,r_effect,black,white,native,asian,hawaiian,other_race,unsure_race,latinx,age,gender
0,1,no,not at all not racist all of the human race is...,False,True,False,False,False,False,False,False,46-55,Man
2,3,i feel that the parole board the some of the p...,not sure about that,False,True,False,False,False,False,False,False,46-55,Woman
3,4,yes bit,i am not really sure about that one,False,True,False,False,False,False,False,False,36-45,Woman
5,6,most are full of,i do not understand this question maybe theres...,True,False,False,False,False,False,False,False,26-35,Woman
6,7,i never considered the prison population or pr...,i try to vote based on the but i am and many i...,False,True,False,False,False,False,False,False,46-55,Woman


In [5]:
# remove stop words
# nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Include the Topic info

In [6]:
def add_the_matrix(extrastopword, csvfile, tlist=["topic 4", "topic 5"]):
    
    assert isinstance(extrastopword, list), "Should be a list of the extra stop words."
    assert isinstance(csvfile, str), "Should be the path of the matrix file."
    
    # change the stop words
    stop_words = stopwords.words("english")
    stop_words.extend(["don", "people", "bill", "step", "act", "first", "u", "n", 
                       "na", "non"]+extrastopword)
    yesnno = ["yes", "no"]
    stop_words =  [word for word in stop_words if word not in yesnno]
    # remove stop words
    r_effect = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in str_data["r_effect"]]
    r_effect = pd.DataFrame([x for x in r_effect if x])
    
    # create index variable for whether a row is left
    r_effect2 = []
    for doc in str_data["r_effect"]:
        if all(word in stop_words for word in simple_preprocess(doc)):
               i_str = False
        else: 
               i_str = True
        r_effect2.append(i_str)
    inBTM = pd.DataFrame(r_effect2, columns=["inBTM"])
    # filter str_data based on inBTM result into str_datar
    str_datar = pd.concat([str_data.reset_index(drop=True), inBTM], axis=1)
    str_datar = str_datar[str_datar["inBTM"]==True]
    
    # read in the topicmatrix file
    matrix = pd.read_csv(csvfile, 
                         names=["topic 1", "topic 2", "topic 3", "topic 4", "topic 5", "topic 6"])
    a = matrix.idxmax(axis=1)
    b = matrix.T.apply(lambda x: x.nlargest(2).idxmin())
    c = matrix.T.apply(lambda x: x.nlargest(3).idxmin())
    topic = pd.concat([a, b, c], axis=1, ignore_index=False)

    # concatenate topic matrix and str_datar as str_datar
    str_datar2 = pd.concat([str_datar.reset_index(drop=True), topic], axis=1)
    str_datar2 = str_datar2.rename(columns={0:"fsttopic", 1:"sndtopic", 2:"trdtopic"})

    # CREATE THE CHISQUARE DATAFRAME
    race = ["black", "white", "native", "asian", "hawaiian", "other_race", "unsure_race", "latinx"]
    topic = ["fsttopic", "sndtopic", "trdtopic"]
    chisq = str_datar2.loc[:, ["Survey_ID"]+race+topic].dropna()
        ## clean the race variable a little bit
    chisq.loc[:, race] = chisq.loc[:, race].astype(int)
    chisq["Race"] = chisq.loc[:, race].idxmax(axis=1)
    
    # CREATE THE REGRESSION DATAFRAME
    chisq["fst4o5"] = chisq["fsttopic"].isin(tlist)
    chisq["snd4o5"] = chisq["sndtopic"].isin(tlist)
    chisq["trd4o5"] = chisq["trdtopic"].isin(tlist)
    cor = chisq.drop(["fsttopic", "sndtopic", "trdtopic", "Race"], axis=1)
    cov = str_data[["Survey_ID", "age", "gender"]]
    cov = pd.get_dummies(cov, columns=["age", "gender"])
    lrdata = cor.merge(cov, on="Survey_ID")
    
    return chisq, lrdata

In [7]:
common_csv = "~/Google Drive/Research Assistant/Work With David Knight/Survey of the Incarcerated/BTM/BTM_Regression_Exploration/"
extra1 = []
csv1 = "All_3095/topicmatrix.csv"

extra2 = ["violent", "race"]
csv2 = "NoViloentRace_3082/topicmatrix_3082.csv"

extra3 = ["violent", "race", "white", "black"]
csv3 = "NoViloentRaceWhiteBlack_3053/topicmatrix_3053.csv"

In [8]:
# All_3095
chisq4, lrdata4 = add_the_matrix(extra1, common_csv+csv1)
# NoViloentRace_3082
chisq5, lrdata5 = add_the_matrix(extra2, common_csv+csv2)
# NoViloentRaceWhiteBlack_3053
# Most Important One
chisq6, lrdata6 = add_the_matrix(extra3, common_csv+csv3)

In [9]:
lrdata5.shape

(3082, 23)

In [10]:
lrdata6.head()

,Survey_ID,black,white,native,asian,hawaiian,other_race,unsure_race,latinx,fst4o5,...,age_26-35,age_36-45,age_46-55,age_56-65,age_66+,age_Under 18,gender_Gender non-conforming or non-binary or other,gender_Man,gender_Prefer not to say,gender_Woman
0,1,0,1,0,0,0,0,0,0,True,...,0,0,1,0,0,0,0,1,0,0
1,3,0,1,0,0,0,0,0,0,True,...,0,0,1,0,0,0,0,0,0,1
2,4,0,1,0,0,0,0,0,0,True,...,0,1,0,0,0,0,0,0,0,1
3,6,1,0,0,0,0,0,0,0,False,...,1,0,0,0,0,0,0,0,0,1
4,7,0,1,0,0,0,0,0,0,True,...,0,0,1,0,0,0,0,0,0,1


In [11]:
lrdata6.to_csv("~/Desktop/lrdata6.csv")

### Clear out all the white samples

In [10]:
def clear_some(dataset):
    dataset_post = dataset[dataset["white"]!=1].drop(["white"], axis=1)
    
    return dataset_post

In [11]:
# All_3095
chisq4, lrdata4 = (df.pipe(clear_some) for df in [chisq4, lrdata4])
# NoViloentRace_3082
chisq5, lrdata5 = (df.pipe(clear_some) for df in [chisq5, lrdata5])
# NoViloentRaceWhiteBlack_3053
# Most Important One
chisq6, lrdata6 = (df.pipe(clear_some) for df in [chisq6, lrdata6])

# Chisq Test

In [12]:
def to_obs(ct):
    obs = np.array([ct.iloc[0][0:7].values,
                    ct.iloc[1][0:7].values,
                    ct.iloc[2][0:7].values,
                    ct.iloc[3][0:7].values,
                    ct.iloc[4][0:7].values,
                    ct.iloc[5][0:7].values,
                    ct.iloc[6][0:7].values,
                    ct.iloc[7][0:7].values])
    return obs

def ctest_race_topic(dataset4chisq):
    ct = pd.crosstab(dataset4chisq.Race, dataset4chisq.fsttopic, margins=True)
    obs = to_obs(ct)
    fst_p = stats.chi2_contingency(obs)[1]
    
    ct = pd.crosstab(dataset4chisq.Race, dataset4chisq.sndtopic, margins=True)
    obs = to_obs(ct)
    snd_p = stats.chi2_contingency(obs)[1]
    
    ct = pd.crosstab(dataset4chisq.Race, dataset4chisq.trdtopic, margins=True)
    obs = to_obs(ct)
    trd_p = stats.chi2_contingency(obs)[1]
    
    result_dic = {"fst_p":fst_p, "snd_p":snd_p, "trd_p":trd_p}
    print(result_dic)
    
    pass

In [13]:
# All_3095
ctest_race_topic(chisq4)
# NoViloentRace_3082
ctest_race_topic(chisq5)
# NoViloentRaceWhiteBlack_3053
ctest_race_topic(chisq6)

{'fst_p': 0.470032583527888, 'snd_p': 0.9104208428669956, 'trd_p': 0.9073542450770896}
{'fst_p': 0.746707133848953, 'snd_p': 0.11252130667121901, 'trd_p': 0.8289105185202188}
{'fst_p': 0.6992108535700728, 'snd_p': 0.2455552954685574, 'trd_p': 0.8313992598916121}


# Linear Regression

In [27]:
lrdata6.columns

Index(['Survey_ID', 'black', 'native', 'asian', 'hawaiian', 'other_race',
       'unsure_race', 'latinx', 'fst4o5', 'snd4o5', 'trd4o5', 'age_18-25',
       'age_26-35', 'age_36-45', 'age_46-55', 'age_56-65', 'age_66+',
       'age_Under 18', 'gender_Gender non-conforming or non-binary or other',
       'gender_Man', 'gender_Prefer not to say', 'gender_Woman'],
      dtype='object')

In [15]:
def reg_on_123(dataset4lr, dropX=[]):
    y1 = dataset4lr["fst4o5"]
    y2 = dataset4lr["snd4o5"]
    y3 = dataset4lr["trd4o5"]

    X = dataset4lr.drop(["Survey_ID", "fst4o5", "snd4o5", "trd4o5", "unsure_race", "age_Under 18",
                         "gender_Gender non-conforming or non-binary or other"]+dropX, axis=1)
    X = sm.add_constant(X)
    
    model1 = sm.OLS(y1, X).fit()
    model2 = sm.OLS(y2, X).fit()
    model3 = sm.OLS(y3, X).fit()
    
    return model1, model2, model3

def return_plist(dataset4lr, dropX=[]):
    model1, model2, model3 = reg_on_123(dataset4lr, dropX)
    
    a = (model1.summary2().tables[1]['P>|t|'] <= 0.05) | (model1.summary2().tables[1]['P>|t|'] >= 0.95)
    a1 = model1.summary2().tables[1]["Coef."]
    b = (model2.summary2().tables[1]['P>|t|'] <= 0.05) | (model2.summary2().tables[1]['P>|t|'] >= 0.95)
    b1 = model2.summary2().tables[1]["Coef."]
    c = (model3.summary2().tables[1]['P>|t|'] <= 0.05) | (model3.summary2().tables[1]['P>|t|'] >= 0.95)
    c1 = model3.summary2().tables[1]["Coef."]
    
    p_df = pd.concat([a1, a, b1, b, c1, c], axis=1)
    p_df.columns = ["fst_inlist_coef", "p<=0.05", "snd_inlist_coef", "p<=0.05", "trd_inlist_coef", "p<=0.05"]
    
    return p_df

### Keep All
These groups of models only excluded some variables to clear perfect multicollinearity.
The variables excluded for perfect multicollinearity include "Survey_ID", "fst1n2", "snd1n2", "trd1n2", "unsure_race","age_Under 18", "gender_Gender non-conforming or non-binary or other".

In [16]:
# All_3095
S3095_all = return_plist(lrdata4)
# NoViloentRace_3082
S3082_all = return_plist(lrdata4)
# NoViloentRaceWhiteBlack_3053
S3053_all = return_plist(lrdata4)

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### No Black
Other than same variables above to clear perfect multicollinearity, these groups of models also excluded "black".

In [17]:
nb = ["black"]

In [18]:
# All_3095
S3095_nb = return_plist(lrdata4, nb)
# NoRace_3082
S3082_nb = return_plist(lrdata5, nb)
# NoRaceWhiteBlack_3053
S3053_nb = return_plist(lrdata6, nb)

In [30]:
print(S3095_nb)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.315607    False         0.440943    False   
native                          -0.033740    False         0.013409    False   
asian                            0.031956    False        -0.013342    False   
hawaiian                        -0.115878    False        -0.183588    False   
other_race                      -0.005656    False        -0.026332    False   
latinx                          -0.038833    False        -0.025335    False   
age_18-25                        0.038797    False         0.050544    False   
age_26-35                        0.030327    False         0.103773    False   
age_36-45                        0.055029    False         0.070134    False   
age_46-55                        0.117291    False         0.059014    False   
age_56-65                        0.057886    False         0.105152    False   
age_66+                         -0.07241

In [29]:
S3095_nb.to_csv("~/Desktop/a.csv")

In [20]:
print(S3082_nb)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.698016     True         0.174647    False   
native                           0.100792     True        -0.074849     True   
asian                           -0.138261    False        -0.002726     True   
hawaiian                        -0.102507    False        -0.118629    False   
other_race                       0.006899    False        -0.052864    False   
latinx                          -0.003950    False         0.067500    False   
age_18-25                       -0.258862    False         0.216146    False   
age_26-35                       -0.216043    False         0.182460    False   
age_36-45                       -0.237426    False         0.183463    False   
age_46-55                       -0.269146    False         0.209129    False   
age_56-65                       -0.247457    False         0.167197    False   
age_66+                         -0.19436

In [21]:
print(S3053_nb)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.853299     True        -0.067230    False   
native                           0.016983    False         0.105932     True   
asian                           -0.035508    False         0.019854    False   
hawaiian                        -0.050005    False         0.021050    False   
other_race                       0.055838    False         0.051842    False   
latinx                          -0.026643    False        -0.025640    False   
age_18-25                       -0.227062    False         0.106393    False   
age_26-35                       -0.323834    False         0.154139    False   
age_36-45                       -0.368702    False         0.227718    False   
age_46-55                       -0.262337    False         0.172522    False   
age_56-65                       -0.288322    False         0.244652    False   
age_66+                         -0.28370

### Only Black
Other than same variables above to clear perfect multicollinearity, these groups of models also excluded all race binary variables excpet "black".

In [22]:
ob = ["native", "asian", "hawaiian", "other_race", "unsure_race", "latinx"]

In [23]:
# All_3095
S3095_ob = return_plist(lrdata4, ob)
# NoRace_3082
S3082_ob = return_plist(lrdata5, ob)
# NoRaceWhiteBlack_3053
S3053_ob = return_plist(lrdata6, ob)

In [24]:
print(S3095_ob)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.236723    False         0.402009    False   
black                            0.055592     True         0.020731    False   
age_18-25                        0.066208    False         0.071407    False   
age_26-35                        0.059067    False         0.124835    False   
age_36-45                        0.084407    False         0.090989    False   
age_46-55                        0.146614    False         0.085227    False   
age_56-65                        0.087942    False         0.128073    False   
age_66+                         -0.036405    False         0.018366    False   
gender_Man                      -0.047842    False        -0.206155    False   
gender_Prefer not to say        -0.189576    False        -0.394010     True   
gender_Woman                    -0.080162    False        -0.194119    False   

                          trd_inlist_co

In [25]:
print(S3082_ob)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.768060     True         0.120787    False   
black                           -0.037982    False        -0.004510    False   
age_18-25                       -0.270322    False         0.239045    False   
age_26-35                       -0.224036    False         0.196337    False   
age_36-45                       -0.244780    False         0.191431    False   
age_46-55                       -0.273849    False         0.223770    False   
age_56-65                       -0.256920    False         0.182635    False   
age_66+                         -0.195966    False         0.200394    False   
gender_Man                      -0.160464    False         0.080115    False   
gender_Prefer not to say        -0.187837    False        -0.001362     True   
gender_Woman                    -0.144815    False        -0.014092    False   

                          trd_inlist_co

In [26]:
print(S3053_ob)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.841611     True         0.027370    False   
black                           -0.023564    False        -0.066778     True   
age_18-25                       -0.192775    False         0.102748    False   
age_26-35                       -0.287422    False         0.153920    False   
age_36-45                       -0.330507    False         0.232219    False   
age_46-55                       -0.226177    False         0.175916    False   
age_56-65                       -0.248657    False         0.244513    False   
age_66+                         -0.241899    False         0.166192    False   
gender_Man                      -0.236898    False         0.185985    False   
gender_Prefer not to say        -0.261418    False         0.274823    False   
gender_Woman                    -0.274334    False         0.168896    False   

                          trd_inlist_co